In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import random
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import os

In [ ]:
url = 'https://www.naukri.com/software-engineering-jobs?k=software%20engineering'

In [11]:
file = 1

for page in range(5,400):  
    driver = webdriver.Chrome()
    
    driver.get('https://www.naukri.com/software-engineering-jobs-{}?k=software+engineering'.format(page))
    
    try:
        elems = WebDriverWait(driver, 5).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.cust-job-tuple.layout-wrapper.lay-2.sjw__tuple'))
        )
        
        print(f"Page {page}: Found {len(elems)} job listings")
        
        with open(f'../scraping/naukri_data/job_listing_{file}.html', 'w', encoding='utf-8') as f:
            for elem in elems:
                d = elem.get_attribute('outerHTML')
                f.write(d)
            file+=1

    except Exception as e:
        print(f"Error on page {page}: {str(e)}")

    driver.quit()


Page 1: Found 19 job listings
Page 2: Found 20 job listings
Page 3: Found 20 job listings
Page 4: Found 19 job listings


Overwrite mode - Do not run

In [13]:
csv_file = open('../data/raw/naukri_job_listings.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Company Name', 'Job Title', 'Experience Required', 'Salary', 'Location', 'Description', 'Skills', 'Time Posted', 'Reviews'])

for i in range(1, 2): 
    driver = webdriver.Chrome()
    
    # Open the desired page
    driver.get(f'https://www.naukri.com/software-engineering-jobs-{i}?k=software+engineering')
    
    try:
        # Wait until the job listings are present in the DOM
        elems = WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.cust-job-tuple.layout-wrapper.lay-2.sjw__tuple'))
        )
        
        print(f"Page {i}: Found {len(elems)} job listings")
        
        # Loop through each job listing
        for elem in elems:
            try:
                job_title = elem.find_element(By.CSS_SELECTOR, 'a.title').text
                company_name = elem.find_element(By.CSS_SELECTOR, 'a.comp-name').text
                experience = elem.find_element(By.CSS_SELECTOR, '.exp-wrap .expwdth').text
                salary = elem.find_element(By.CSS_SELECTOR, '.sal-wrap span').text
                location = elem.find_element(By.CSS_SELECTOR, '.loc-wrap .locWdth').text
                description = elem.find_element(By.CSS_SELECTOR, '.job-desc').text
                time_posted = elem.find_element(By.CSS_SELECTOR, '.job-post-day').text
                
                skills = ', '.join([skill.text for skill in elem.find_elements(By.CSS_SELECTOR, '.tags-gt .tag-li')])
                
                try:
                    reviews = elem.find_element(By.CSS_SELECTOR, '.review').text
                except:
                    reviews = 'No reviews'
                
                csv_writer.writerow([company_name, job_title, experience, salary, location, description, skills, time_posted, reviews])

            except Exception as e:
                print(f"Error extracting details for one of the job listings: {str(e)}")
    
    except Exception as e:
        print(f"Error on page {i}: {str(e)}")

    driver.quit()

csv_file.close()

Page 1: Found 19 job listings
Page 2: Found 20 job listings
Page 3: Found 19 job listings
Error extracting details for one of the job listings: Message: no such element: Unable to locate element: {"method":"css selector","selector":".exp-wrap .expwdth"}
  (Session info: chrome=128.0.6613.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF63010B632+29090]
	(No symbol) [0x00007FF63007E6E9]
	(No symbol) [0x00007FF62FF3B1CA]
	(No symbol) [0x00007FF62FF8EFD7]
	(No symbol) [0x00007FF62FF8F22C]
	(No symbol) [0x00007FF62FF827CC]
	(No symbol) [0x00007FF62FFB672F]
	(No symbol) [0x00007FF62FF826A6]
	(No symbol) [0x00007FF62FFB6900]
	(No symbol) [0x00007FF62FFD65D9]
	(No symbol) [0x00007FF62FFB6493]
	(No symbol) [0x00007FF62FF809B1]
	(No symbol) [0x00007FF62FF81B11]
	GetHandleVerifier [0x00007FF63042881D+3294093]
	GetHandleVerifier [0x00007FF630474403+3604339]

Append the data and run multiple times

In [3]:
def safe_extract_text(element, selector, attribute='text'):
    try:
        if attribute == 'text':
            return element.find_element(By.CSS_SELECTOR, selector).text.strip()
        else:
            return element.find_element(By.CSS_SELECTOR, selector).get_attribute(attribute).strip()
    except:
        return "no data available"

fields = {
    'Job Title': ('.title', 'text'),
    'Company Name': ('.comp-name', 'text'),
    'Experience Required': ('.exp-wrap .expwdth', 'text'),
    'Salary': ('.sal-wrap .ni-job-tuple-icon-srp-rupee', 'text'),
    'Location': ('.loc-wrap .locWdth', 'text'),
    'Description': ('.job-desc', 'text'),
    'Skills': ('.tags-gt .tag-li', 'list'),
    'Time Posted': ('.job-post-day', 'text'),
    'Reviews': ('.review', 'text')
}

csv_file_path = '../data/raw/naukri_job_listings.csv'
file_exists = os.path.isfile(csv_file_path)

with open(csv_file_path, mode='a' if file_exists else 'w', newline='', encoding='utf-8') as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=fields.keys())
    
    if not file_exists:
        writer.writeheader()

    driver = webdriver.Chrome()

    for page in range(1501, 1502):  # Loop through pages
        driver.get(f'https://www.naukri.com/software-engineering-jobs-{page}?k=software+engineering')
        
        try:
            job_listings = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.cust-job-tuple.layout-wrapper.lay-2.sjw__tuple'))
            )
            print(f"Page {page}: Found {len(job_listings)} job listings")
            
            for listing in job_listings:
                job_data = {}
                
                for field, (selector, attr) in fields.items():
                    if attr == 'list':
                        job_data[field] = ', '.join([item.text for item in listing.find_elements(By.CSS_SELECTOR, selector)]) or "no data available"
                    else:
                        job_data[field] = safe_extract_text(listing, selector, attr)
                
                writer.writerow(job_data)
                
        except Exception as e:
            print(f"Error extracting job listings on page {page}: {str(e)}")

    driver.quit()

Page 1501: Found 20 job listings
